In [17]:
!kaggle competitions download -c titanic -p "./dataset/"

100%|███████████████████████████████████████| 34.1k/34.1k [00:00<00:00, 263kB/s]
100%|███████████████████████████████████████| 34.1k/34.1k [00:00<00:00, 263kB/s]


In [16]:
!unzip ./dataset/titanic.zip -d ./dataset && rm ./dataset/titanic.zip

Archive:  ./dataset/titanic.zip
  inflating: ./dataset/gender_submission.csv  
  inflating: ./dataset/test.csv      
  inflating: ./dataset/train.csv     


In [2]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [79]:
train_filepath = "./dataset/train.csv"
test_filepath = "./dataset/test.csv"
gender_submission_filepath = "./dataset/gender_submission.csv"
train = pd.read_csv(train_filepath)
test = pd.read_csv(test_filepath)
gender = pd.read_csv(gender_submission_filepath)

In [80]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


결측치 보정

In [81]:
print(train.columns)
print(train.shape)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
(891, 12)


In [82]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Cabin 칼럼의 경우 결측치의 비중이 너무 높으므로 남은 데이터로 추정하면 노이즈만 증가할 것으로 보이므로 제거

In [83]:
train_thresh = train.dropna(axis = 1, thresh = 2*len(train)/3)


In [84]:
train_thresh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(4)
memory usage: 76.7+ KB


In [76]:
# df = train[train["Embarked"].isnull() | train["Age"].isnull() | train["Cabin"].isnull()]
# df

In [85]:
train_thresh.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         2
dtype: int64

In [86]:
train_thresh["Embarked"].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [89]:
train_thresh = train_thresh[train_thresh["Embarked"].isnull()].fillna("S")

In [90]:
train_thresh[train_thresh["Embarked"].isnull()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked


In [91]:
# print(train.Survived.value_counts())

In [96]:
# data.replace("male", 0, inplace=True); data.replace("female",1,inplace=True)
train['Sex'] = train["Sex"].astype('category')
# print(train.head(2))
print(train.Sex.cat.categories)
print(train.Sex.cat.codes.head(5))

train["Sex"] = train.Sex.cat.codes
train["Sex"]

Int64Index([0, 1], dtype='int64')
0    1
1    0
2    0
3    0
4    1
dtype: int8


0      1
1      0
2      0
3      0
4      1
      ..
886    1
887    0
888    0
889    1
890    1
Name: Sex, Length: 891, dtype: int8

In [28]:
sns.heatmap(data)

ValueError: could not convert string to float: 'Kelly, Mr. James'